<img src="../images/felipepelaquim-9c7CK4BemmA-unsplash.jpg"  style="float: center; margin: 0px; height: 350px; width: 1000px"> 

# Afrobeats influence in Hip Hop

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import time

In [2]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id = '2101cd224f5948e19c4c782d76744ed3',
                                                      client_secret = '879abdfca432449facc9d8566fb40ab6')

Create spotipy object using SpotifyClientCredentials

In [3]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
# results = sp.playlist_tracks('5ZCzd0nCLqiIX1jwQWfazW')
# tracks = results['items']
    
# while results['next']:
#     results = sp.next(results)
#     tracks.extend(results['items'])

# tracks[4]['track'].keys()

# tracks[4]['track']['uri']

## Get song info

Create a function that takes in a playlist of songs and extracts the songs information and features such as artist name, genre, album name, track_uri, danceability, energy, loudness, instrumentalness etc

In [5]:
def get_track_info(playlist):
    
    #split the playlist_uri is at the end of the playlists url. I'll use the .split method to extract it
    uri = playlist.split("/")[-1].split("?")[0]
    
    #from the spotipy library, use the playlist_tracks() method to extract each track from the playlist uri
    #It comes in a nested dictionary format
    
    results = sp.playlist_tracks(uri)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    
    #create an empty dictionary with the info we want to extract as columns
    info = {
    'track_uri':[],
    'track_name':[],
    'artist_name':[],
    #'artist_info':[],
    'artist_uri':[],
    #'artist_popularity':[],
    #'artist_genre':[],
    'album':[],
    'track_pop':[],
    }
    
    features = {'danceability': [],
     'energy': [],
     'key': [],
     'loudness': [],
     'mode': [],
     'speechiness': [],
     'acousticness': [],
     'instrumentalness': [],
     'liveness': [],
     'valence': [],
     'tempo': [],
     'type': [],
     'id': [],
     'uri': [],
     'track_href': [],
     'analysis_url': [],
     'duration_ms': [],
     'time_signature': []
               }
    
    #using a for loop, get the the info for each song and put it into the empty dictionary
    for track in tracks:
        #URI
        info['track_uri'].append((track["track"]["uri"]).split(':')[2])

        #Track name
        info['track_name'].append(track["track"]["name"])

        #Main Artist
        info['artist_uri'].append((track["track"]["artists"][0]["uri"]).split(':')[2])
        #info['artist_info'].append(sp.artist(track["track"]["artists"][0]["uri"]))

        #Name, popularity, genre
        info['artist_name'].append(track["track"]["artists"][0]["name"])
        #info['artist_popularity'].append(sp.artist(track["track"]["artists"][0]["uri"])["popularity"])
        #info['artist_genre'].append(sp.artist(track["track"]["artists"][0]["uri"])["genres"])

        #Album
        info['album'].append(track["track"]["album"]["name"])

        #Popularity of the track
        info['track_pop'].append(track["track"]["popularity"])
        
        #Transform the info dictionary into a dataframe
        info_df = pd.DataFrame(info)
        
        #loop through the tracks to their features and assign it to the empty dictionary
        track_uri = track["track"]["uri"].split(':')[2] 
        
        for key,value in (sp.audio_features(track_uri)[0]).items():
            features[key].append(value)
            
        #time.sleep(1)
        #time.sleep(random.uniform(1,5))
        
    #Transform the features dictionary into a dataframe
    features_df = pd.DataFrame(features)
    
    #drop features we do not need
    #features_df.drop(columns=['type','id','uri','track_href','analysis_url','duration_ms','time_signature'], inplace=True)   
    
    
    
     
    
    return info_df.join(features_df)
        

#### Create dataframe of users playlist

#### Afrobeats playlist

In [7]:
def raw_data(user_playlist_url, genre):
    user_playlist_info = get_track_info(user_playlist_url)
    #add user genre
    user_playlist_info.loc[:,'genre'] = genre
    return user_playlist_info

In [8]:
afrobeats_playlist_url= "https://open.spotify.com/playlist/5ZCzd0nCLqiIX1jwQWfazW"

In [9]:
afrobeats_df = raw_data(afrobeats_playlist_url, 'afrobeats')

In [10]:
afrobeats_df.head()

,track_uri,track_name,artist_name,artist_uri,album,track_pop,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,1sSt1DqqqFLkPwfrqafVyn,Oleku (feat. Brymo),51,0.544,0.730,1,-4.299,...,0.751,162.112,audio_features,0mDRuZmReEm6DquPLJlcEm,spotify:track:0mDRuZmReEm6DquPLJlcEm,https://api.spotify.com/v1/tracks/0mDRuZmReEm6...,https://api.spotify.com/v1/audio-analysis/0mDR...,291364,5,afrobeats
1,1tvi8tv0eykhNcV1WtaIqO,Move Back,5five,37zb1JQnDV9dRLatrASEj1,Move Back,45,0.793,0.734,11,-9.252,...,0.793,125.032,audio_features,1tvi8tv0eykhNcV1WtaIqO,spotify:track:1tvi8tv0eykhNcV1WtaIqO,https://api.spotify.com/v1/tracks/1tvi8tv0eykh...,https://api.spotify.com/v1/audio-analysis/1tvi...,237107,4,afrobeats
2,5YgyvTZmdkIfFOnj8ri8mZ,Pakurumo,WizKid,3tVQdUvClmAT7URs9V3rsp,Superstar,49,0.879,0.737,2,-5.412,...,0.967,119.990,audio_features,5YgyvTZmdkIfFOnj8ri8mZ,spotify:track:5YgyvTZmdkIfFOnj8ri8mZ,https://api.spotify.com/v1/tracks/5YgyvTZmdkIf...,https://api.spotify.com/v1/audio-analysis/5Ygy...,212006,4,afrobeats
3,2cWdhA4ZDBvYuHRKKIPT3k,The Matter (feat. Wizkid),Maleek Berry,520qA5VGL9iI0SbmEnTVNg,The Matter (feat. Wizkid),49,0.697,0.841,2,-6.782,...,0.808,103.070,audio_features,2cWdhA4ZDBvYuHRKKIPT3k,spotify:track:2cWdhA4ZDBvYuHRKKIPT3k,https://api.spotify.com/v1/tracks/2cWdhA4ZDBvY...,https://api.spotify.com/v1/audio-analysis/2cWd...,198058,4,afrobeats
4,4voOsb8qpJ569JaMiGAyrl,Slow Down (feat. Wiz Kid),R2Bees,0LFsP7WPfu5inz9a1amcE4,Refuse to Be Broke: Da Revolution 2,53,0.520,0.717,6,-3.006,...,0.704,135.550,audio_features,4voOsb8qpJ569JaMiGAyrl,spotify:track:4voOsb8qpJ569JaMiGAyrl,https://api.spotify.com/v1/tracks/4voOsb8qpJ56...,https://api.spotify.com/v1/audio-analysis/4voO...,261547,5,afrobeats


In [11]:
len(afrobeats_df)

1500

#### Jacks playlist

In [13]:
jacks_playlist_url = 'https://open.spotify.com/playlist/6UlskZAcTPzcGMnQaMnIVm?si=5cbb031d1fdd4064'

In [14]:
jacks_playlist_df = raw_data(jacks_playlist_url, 'jack')

#### Ankita playlist

In [15]:
ankita_playlist_url = 'https://open.spotify.com/playlist/6qzbkhrmxdFj5TtaXR0sfI?si=ElY40mMjQ7apOsHTGlQI7A'

In [16]:
ankita_playlist_df = raw_data(ankita_playlist_url, 'ankita')

#### Export dataframes as a csv

In [17]:
afrobeats_df.to_csv('../data/afrobeats.csv', index = False)

In [18]:
jacks_playlist_df.to_csv('../data/jack.csv', index = False)

In [19]:
ankita_playlist_df.to_csv('../data/ankita.csv', index = False)